# Module 3 Hierarchical Clustering

We have seen how Kmeans algorithm formed clusters which expects pre determined number of clusters to be formed. In this notebook we will move on to hierarchical clustering which has the benefit that one doesn't need to already know the number of clusters k in data. We will use the same old iris data to illustrate the concept in this notebook.

Read the iris data from /datasets folder

In [ ]:
import scipy.cluster.hierarchy as hca
import pandas as pd
import numpy as np
from pylab import *
from matplotlib import pyplot
%matplotlib inline
np.set_printoptions(suppress=True)

data=pd.read_csv("../../../datasets/iris.txt")
data.head(1)

There is an extra column of data. Get rid of it.  

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)
num_data = data.iloc[:,:-1]

In [ ]:
plt.scatter(num_data.iloc[:,0], num_data.iloc[:,1])
plt.show()

In [ ]:
# generate the linkage matrix
Z = hca.linkage(data.iloc[:,:-1], 'ward')

**`ward`** is one of the methods that can be used to calculate the distance between newly formed clusters. 'ward' causes linkage() to use the Ward variance minimization algorithm.

There are some other common linkage methods like 'single', 'complete', 'average' along with different distance metrics like 'euclidean' (default), 'Manhattan', 'hamming', 'cosine'. You can try different distance measures clustering methods to make sure that the data is not just be clustered to minimize the overall intra cluster variance in euclidean space. For example, when dealing with clustering text documents, it may not look right to use euclidean distance metric with long (binary) feature vectors (word-vectors in text clustering).

Explore the linked methods and metrics to make a informed choice between different options available. 

**`Cophenetic Correlation Coefficient`:** Check the Cophenetic Correlation Coefficient of the clustering generated using cophenet() function. This compares (correlates) the actual pairwise distances of all data samples to those implied by the hierarchical clustering. The closer the value is to 1, the better the clustering preserves the original distances.

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, cop_dist = cophenet(Z, pdist(num_data))
c

The linkage() function will use the supplied method and metric to calculate the distances of the clusters starting with all the **`n`** individual data points as singleton clusters and in each iteration will merge the two clusters which have the smallest distance according the selected method and metric. 

It will return an array of length n - 1 giving the information about the n - 1 cluster merges which it did to pairwise merge the n singleton clusters. Z[i] will tell which clusters were merged in the i-th iteration as shown below.

In [ ]:
Z[0]

So each row of the resulting array has the format [idx1, idx2, dist, sample_count]. Rows 101 and 142 in the dataset are clustered together to form first cluster. These two samples have 0 distance and created a cluster with a total of 2 samples.

In [ ]:
Z[1]

In [ ]:
Z[2]

In second iteration, rows 7 and 39 in the dataset are clustered together to form a cluster. These two samples have a distance of 0.1. Have a look at the first 20 iterations.

In [ ]:
Z[:30]

If you can observe the sample_count column, until iteration 22 the algorithm only directly merged original samples. Also, there has been a monotonic increase in the distance.

In iteration 22 the algorithm merged cluster indices 40 with 152. Do you remember that teher are only 150 samples in the dataset meaning the original sample indices are 0 to 149 for 150 samples. All indices idx >= len(num_data) actually refer to the cluster formed in Z[idx - len(num_data)].

Simply put, while idx 149 corresponds to num_data[149], idx 150 corresponds to the cluster formed in Z[0], idx 151 to Z[1], 152 to Z[2] and so on...

Hence, the merge iteration 22 merged sample 40 to samples 0 and 17 that were previously merged in iteration 2 (152 - 2).

Let's check out the points coordinates to see if this makes sense:

In [ ]:
num_data.loc[[0, 17, 40], :]

The points seem pretty close. Plot the points highlighted to verify them.

In [ ]:
# PLotting Sepal.Length vs Petal.Width
idxs = [0, 17, 40]
plt.figure(figsize=(10, 8))
plt.scatter(num_data.iloc[:,0], num_data.iloc[:,3])  # plot all points

# Highlight the points of interest in Red color
plt.scatter(num_data.iloc[idxs,0], num_data.iloc[idxs,3], c='r')  # plot interesting points in red again
plt.show()

All 3 red dots are pretty close to each other as predicted. Lets check the results of 28th iteration. The algorithm merged indices 88 to 82 and 92. 

In [ ]:
num_data.loc[[88, 82, 92], :]

In [ ]:
# PLotting Sepal.Length vs Petal.Width
idxs = [88, 82, 92]
plt.figure(figsize=(10, 8))
plt.scatter(num_data.iloc[:,0], num_data.iloc[:,1])  # plot all points

# Highlight the points of interest in Red color
plt.scatter(num_data.iloc[idxs,0], num_data.iloc[idxs,1], c='r')  # plot interesting points in red again
plt.show()

### Plot the Dendogram

A dendrogram is a visualization technique that is in the form of a tree showing the order and distances of merges during the hierarchical clustering.

In [ ]:
y=hca.linkage(num_data,metric="euclidean",method="ward")

**Reference: ** 
- [scipy.cluster.hierarchy.linkage(y, method='single', metric='euclidean')](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage). 
 
linkage() performs hierarchical/agglomerative clustering on the condensed distance matrix y. y must be a ${n \choose 2}$ sized vector where n is the number of original observations paired in the distance matrix. 

In below code block, truncate_mode='lastp' option tells python to show just the last p non-singleton clusters formed in the linkage. All other non-singleton clusters are contracted into leaf nodes. 

The number p indicates number of clusters to be shown. Its gives a truncated dendrogram, which only shows the last p=25 out of the 149 merges.

In [ ]:
plt=hca.dendrogram(y,truncate_mode="lastp",p=25)
xticks(rotation=90)
ylabel("Distance")
figtext(0.5,0.95,"Iris flower data set",ha="center",fontsize=12)
figtext(0.5,0.90,"Dendrogram (center, euclidean, ward)",ha="center",fontsize=10)

**Reference: **

- [scipy.cluster.hierarchy.dendrogram(data,....)](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram)

Plots the hierarchical clustering as a dendrogram. The dendrogram illustrates how each cluster is composed by drawing a U-shaped link between a non-singleton cluster and its children. The height of the top of the U-link is the distance between its children clusters. It is also the cophenetic distance between original observations in the two children clusters. It is expected that the distances in data[:,2] be monotonic, otherwise crossings appear in the dendrogram.

- The x axis labels, if nothing is specified then they are the indices of samples in num_data.
- The labels on y axis represent the distances (of the 'ward' method in this case).

Interpreting the dendogram:

Starting from each label at the bottom, you can see a vertical line up to a horizontal line. The height of that horizontal line tells you about the distance at which this label was merged into another label or cluster. 

Summarizing:

Horizontal lines are cluster merges. Vertical lines tell you which clusters/labels were part of merge forming that new cluster. Heights of the horizontal lines tell you about the distance that needed to be "bridged" to form the new cluster. You can also see that from distances > 12 up there's a huge jump of the distance to the final merge at a distance of approx 33. Let's have a look at the distances of the last 4 merges:

In [ ]:
Z[-4:,2]

Such merges in the dendrogram are interesting. They indicate that something has been merged that shouldn't be merged may be. Simply put, maybe the things that were merged really don't belong to the same cluster suggesting that maybe there are just 2 clusters different clusters.

The colors of the clusters come from color_threshold argument of dendrogram(). Because it is not specified here, it automatically picked a distance cut-off value of 70 % of the final merge and then colored the clusters.

### Selecting a Distance Cut-Off or Determining the Number of Clusters

As explained above already, a huge jump in distance is typically what we're interested in when arguing for a certain number of clusters. When you have to pick the number of clusters manually, that is probably the best method as it allows to gain some insights into the data and to perform some sanity checks on the edge cases. In this case the distance cut off could be 10, as the jump is pretty obvious. 

When automatically picking the number of clusters there are a couple of common methods that can be used. 

#### Inconsistency Method
"inconsistency" method is one of the defaults for the fcluster() function in scipy.

The question driving the inconsistency method is "what makes a distance jump a jump?". It answers this by comparing each cluster merge's height h to the average avg and normalizing it by the standard deviation. Link statistics are computed over link heights for links d levels below the cluster i.

$$inconsistency=\frac{h−avg}{std}$$
The following shows a matrix of the avg, std, count, inconsistency for each of the last 10 merges of our hierarchical clustering with depth = 5

In [ ]:
from scipy.cluster.hierarchy import inconsistent

depth = 5
incons = inconsistent(Z, depth)
incons[-10:]

There is no paricular golden rule to pick the depth limit in the inconsistencies. See what happens if depth is set to 3 instead.

In [ ]:
depth = 3
incons = inconsistent(Z, depth)
incons[-10:]

The inconsistency values heavily depend on the depth of the tree you calculate the averages over. 

#### Elbow Method

It tries to find the clustering step where the acceleration of distance growth is the biggest.

In [ ]:
print(last)
print(acceleration)
print(idxs)
print(idxs[:-2])
print(acceleration_rev)

In [ ]:
import matplotlib.pyplot as plt

# Take last 10 merge distances into variable last
last = Z[-10:, 2]

# reverse the elemensts in variable last and assign them to last_rev. The index '-1' starts the indexing from the last.
last_rev = last[::-1]

# create an array with indexes 1,2,3...10 i;e start from 1 to until 11. It will not include 11. 
idxs = np.arange(1, len(last) + 1)

# Plot the merge distances. last_rev has merge with highest value at index 0. 
plt.plot(idxs, last_rev)


# Calculate the 2nd order discrete difference along given axis.
acceleration = np.diff(last, 2)  # 2nd derivative of the distances

# reverse the elemensts in variable acceleration and assign them to acceleration_rev. 
acceleration_rev = acceleration[::-1]

# Plot the 2nd order distances. 
plt.plot(idxs[:-2] + 1, acceleration_rev)
plt.show()

# Find the index of largest value in acceleration_rev to decide the number of clusters. 
k = acceleration_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters
print("clusters:", k)

### Retrieve the Clusters

For retrieving the clusters, for different ways of determining k the fcluster function is used.

### Knowing max_d:

Onvce the max distance is determined with help of a dendrogram, then cluster id for each of the samples can be extracted using below peice of code.

In [ ]:
from scipy.cluster.hierarchy import fcluster
max_d = 50
clusters = fcluster(Z, max_d, criterion='distance')
clusters

#### Knowing k:

In [ ]:
k=2
fcluster(Z, k, criterion='maxclust')

#### Using the Inconsistency Method (default):

Inconsistency Method is the default method in fcluster function. 

In [ ]:
from scipy.cluster.hierarchy import fcluster
clusters=fcluster(Z, 8, depth=10)
clusters

#### Visualizing Your Clusters

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(num_data.iloc[:,0], num_data.iloc[:,1], c=clusters, cmap='prism')  # plot points with cluster dependent colors
plt.show()